In [16]:
import pyspark as ps
import pandas as pd
import numpy as np
import pickle
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.metrics import mean_squared_error
spark = SparkSession.builder.getOrCreate()

In [2]:
df = pd.read_csv('data/filtered_ratings.csv')
# data = data[['user', 'movie', 'rating']]

In [3]:
df.shape

(22826086, 7)

In [4]:
data = df.sample(frac=0.15, random_state=51)

In [5]:
data = data.drop(['count','mean','std','stat_score'], axis=1)
data = data.dropna()

In [6]:
ratings_df = spark.createDataFrame(data)

In [7]:
train, test = ratings_df.randomSplit([0.8, 0.2], seed=51)

In [8]:
train.count()

2739331

In [9]:
# density of my training data
# num_ratings/ (num_users * num_movies)
num_rantings = train.count()
num_users = train.select("userId").distinct().count()
num_movies = train.select("movieId").distinct().count()

density = num_rantings/ (num_users * num_movies)
density

0.0010023206103166764

In [10]:
# create an untrained ALS factorization model.
from pyspark.ml.recommendation import ALS
als_model = ALS(
    itemCol='movieId',
    userCol='userId',
    ratingCol='rating',
    nonnegative=True,
    coldStartStrategy="drop",
    maxIter=20,
    regParam=0.2) 

In [11]:
recommender = als_model.fit(train)

In [12]:
predictions = recommender.transform(test)

In [13]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 38679|    148|   3.0|   2.20292|
| 98520|    148|   4.0| 2.4361868|
|145182|    148|   3.0| 2.2352624|
|115912|    148|   3.0| 2.4457183|
| 28366|    148|   5.0| 3.1746926|
| 93292|    148|   3.0| 2.6864526|
|139509|    471|   3.0| 3.6501803|
| 89882|    471|   3.0| 3.3075888|
| 95398|    471|   2.0| 1.8233005|
|106346|    471|   5.0|  3.327161|
| 35901|    471|   5.0| 4.1983585|
| 75881|    471|   4.0|  1.515373|
| 53344|    471|   5.0| 3.7850823|
|142682|    471|   3.5| 3.1262941|
|111641|    471|   3.5| 3.8141546|
| 27128|    471|   4.0| 3.2349274|
|109522|    471|   4.0| 3.3698845|
|135692|    471|   4.0|  4.484868|
| 67762|    471|   3.0| 2.9312148|
| 81672|    471|   5.0| 3.4823003|
+------+-------+------+----------+
only showing top 20 rows



In [25]:
# save best model to specified path
mPath =  "als_recommender"
recommender.write().overwrite().save(mPath)

In [35]:
# read pickled model via pipeline api
from pyspark.ml import recommendation
persistedModel = recommendation.ALSModel.load(mPath)

In [36]:
# predict
predictionsDF = persistedModel.transform(test)

In [37]:
predictionsDF.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     3|  60684|   4.5|  3.710263|
|     4| 148626|   4.0|  3.611896|
|     8|    333|   5.0|  3.047336|
|    12|   2082|   2.5|  2.611706|
|    12|   2967|   3.0|  3.474644|
|    13|    367|   4.0| 3.2134213|
|    13|   4161|   3.5| 3.1478963|
|    19|  45722|   2.0|  2.940666|
|    19| 101962|   2.5| 3.5853627|
|    23|   3108|   5.0| 3.8588307|
|    26|   3173|   3.5| 3.1618593|
|    31|   2502|   3.0| 2.5100248|
|    31|   4963|   4.0| 2.3915906|
|    31|   5902|   2.5| 2.4427898|
|    31|  55765|   4.0| 2.4065835|
|    36|    500|   4.0| 2.9858372|
|    38|   4878|   5.0|  3.578259|
|    38|   6947|   4.0| 3.3136005|
|    41|    858|   4.0| 3.4797316|
|    43|   4873|   3.5| 3.4248993|
+------+-------+------+----------+
only showing top 20 rows



In [22]:
rec2 = ml.read.load('als_recommender')

NameError: name 'ml' is not defined

In [17]:
predictions_pd = predictions.toPandas()
y = np.array(predictions_pd.rating.values)
y_hat = np.array(predictions_pd.prediction.values)

In [18]:
MSE = mean_squared_error(y, y_hat)
RMSE = np.sqrt(MSE)
MSE, RMSE

(0.9794012574943147, 0.9896470368238944)

In [19]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
evaluator.evaluate(predictions)

Py4JJavaError: An error occurred while calling o153.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 248.0 failed 1 times, most recent failure: Lost task 9.0 in stage 248.0 (TID 1995, 26701612e8e0, executor driver): java.lang.StackOverflowError
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1821)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2117)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1646)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2412)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2306)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2144)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1646)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:464)
	at java.base/java.io.ObjectInputStrea

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 59980)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [32]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.9432096885574339
